# PG Fixed evader trajectory, fixed pursuer initial position with Master training

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys

import torch
from torch import optim

print(sys.version)
print(torch.__version__)
print(torch.version.cuda)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(device)

3.9.5 (default, Nov 18 2021, 16:00:48) 
[GCC 10.3.0]
1.9.1+cu111
11.1
cuda


## Environment

In [2]:
from environment import Environment

sel_env = 1
res = 1

env_file = open('Env'+str(sel_env)+'_res'+str(res)+'.txt', 'r')
env_head = np.fromstring(env_file.readline(), dtype=np.int32, sep=' ')
env_file.close()

env = Environment(env_head, device)

## Create fully connected network

In [3]:
from models import pursuer_NN
pursuer_model_training      = pursuer_NN(input_size=4, n_actions=5, num_neurons_hidden=256, device=device).to(device)
pursuer_model               = pursuer_NN(input_size=4, n_actions=5, num_neurons_hidden=256, device=device).to(device)

master  = pursuer_NN(input_size=4, n_actions=5, num_neurons_hidden=256, device=device).to(device)
master  = torch.load('pursuer_NN_Env'+str(sel_env)+'_res'+str(res)+'.pt')
master.eval()

pursuer_NN(
  (predict): Sequential(
    (0): Linear(in_features=4, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=256, bias=True)
    (7): ReLU()
    (8): Linear(in_features=256, out_features=5, bias=True)
    (9): Softmax(dim=-1)
  )
)

In [4]:
def discount_rewards(rewards, gamma=0.99):
    r = np.array([gamma**i * rewards[i] for i in range(len(rewards))])

    # Reverse the array direction for cumsum and then
    # revert back to the original order
    r = r[::-1].cumsum()[::-1]

    return r#(r - r.mean())# / (r.std() + np.finfo(np.float32).eps)

In [5]:
def REINFORCE_PURSUER_FIXED(total_trajectories=2000, gamma=0.9, lr=0.001, batch_size=8, MAX_EPSILON = 1.0, MIN_EPSILON = 0.1, DECAY_RATE = 0.001):
    # Set up lists to hold results
    train_rewards  = []
    train_rewards.append(0)
    test_rewards   = []
    test_rewards.append(0)
    test_episodes  = []
    test_episodes.append(0)

    batch_states            = []
    batch_rewards           = []
    batch_total_rewards     = []
    batch_discount_actions  = []
    batch_counter           = 0


    optimizer = optim.Adam(pursuer_model_training.parameters(), lr=lr)

    for ep in range(total_trajectories):
        s_0 = env.reset_fixed_initial_position()

        states          = []
        actions         = []
        rewards         = []

        done = False
        t = 0

        
        while not done:
            epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * ep)
            if np.random.rand() < epsilon:
                p_dist = master(s_0).detach().cpu().numpy()
            else:
                p_dist = pursuer_model_training(s_0).detach().cpu().numpy()

            
            s_1, r, done, a_p = env.step_training(p_dist, t)
            
            states.append(s_0)
            rewards.append(r)
            actions.append(a_p)

            t += 1
            s_0 = s_1

            if done:
                batch_states.extend(states)
                batch_rewards.extend(discount_rewards(rewards, gamma))
                batch_total_rewards.append(sum(rewards))
                batch_discount_actions.extend(actions)

                batch_counter += 1

                if batch_counter ==  batch_size:
                    optimizer.zero_grad()

                    state_tensor    = torch.FloatTensor(batch_states)
                    reward_tensor   = torch.FloatTensor(batch_rewards).to(device)
                    action_tensor   = torch.LongTensor(batch_discount_actions)

                    # Calculate loss
                    logprob             = torch.log(pursuer_model_training(state_tensor)).to(device)
                    selected_logprobs   = reward_tensor * logprob[np.arange(len(action_tensor)), action_tensor]
                    loss                = -selected_logprobs.mean()

                    # Calculate gradients
                    loss.backward()

                    # Apply gradients
                    optimizer.step()

                    batch_rewards   = []
                    batch_discount_actions   = []
                    batch_states    = []
                    batch_counter   = 0

                    # Test the neural network with the best action
                    s_0 = env.reset_fixed_initial_position()
                    rewards_test = 0
                    done = False
                    t = 0
                    while not done:
                        p_dist = pursuer_model_training(s_0).detach().cpu().numpy()
                        s_1, r, done, _ = env.step_best_action(p_dist, t)
                        rewards_test += r
                        t += 1
                        s_0 = s_1
                    
                    if rewards_test > test_rewards[-1]:
                        pursuer_model.load_state_dict(pursuer_model_training.state_dict())
                        test_rewards.append(rewards_test)
                    else:
                        test_rewards.append(test_rewards[-1])
                        
                    train_rewards.append(np.mean(batch_total_rewards))
                    
                    batch_total_rewards = []
                    test_episodes.append(ep)

                    print('\rEp: {}, average training rewards {:.4f}, test rewards {:.4f} '.format(ep+1, train_rewards[-1], test_rewards[-1]), end='')

    return test_episodes, train_rewards, test_rewards

In [6]:
test_episodes, train_rewards, test_rewards = REINFORCE_PURSUER_FIXED(total_trajectories=12000, batch_size=12, lr=0.0005, MIN_EPSILON=0.1)

Ep: 4404, average training rewards 8.0000, test rewards 8.0000 

KeyboardInterrupt: 

In [ ]:
fig = plt.figure(figsize=(18,8))
ax = fig.add_subplot(1, 1, 1)
plt.plot(test_episodes, train_rewards, label='Mean reward in training at the end of batch')
plt.plot(test_episodes, test_rewards, label='Reward with best policy at the end of batch')

window = 10
smoothed_train_rewards = [np.mean(train_rewards[i-window:i+1]) if i > window 
                    else np.mean(train_rewards[:i+1]) for i in range(len(train_rewards))]
plt.plot(test_episodes, smoothed_train_rewards, 'k')

plt.xlabel('Number of trajectories in training')
plt.ylabel('Reward')
plt.title('Training mean rewards and best policy rewards')
plt.legend()
#grid
yticks = np.linspace(0, test_rewards[-1], test_rewards[-1]+1)
ax.set_yticks(yticks)
plt.grid()
plt.savefig('pursuer_fixed_initial_position_Supervisor_' + str(sel_env) + '.png')
plt.show()

In [ ]:
torch.save(pursuer_model,  'pursuer_RL_supervisor_Env' + str(sel_env) + '_res' + str(res) + '.pt')
torch.save(pursuer_model.state_dict(),  'pursuer_dict_RL_supervisor_Env' + str(sel_env) + '_res' + str(res) + '.pt')

np.savetxt('pursuer_RL_Env' + str(sel_env) + '_res' + str(res) + '_train_rewrds_e-greedy_Supervisor.txt', train_rewards, fmt='%.4f')
np.savetxt('pursuer_RL_Env' + str(sel_env) + '_res' + str(res) + '_test_rewrdse-greedy_Supervisor.txt', test_rewards, fmt='%.4f')